## import library

In [2]:
import tensorflow as tf
import sklearn

print("TensorFlow version:", tf.__version__)
print("Scikit-learn version:", sklearn.__version__)


TensorFlow version: 2.18.0
Scikit-learn version: 1.6.1


In [3]:
# Step 1: Import dan baca data
import pandas as pd
import numpy as np
import tensorflow as tf
# 2. Encoding user & place
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Input
from sklearn.model_selection import train_test_split

## load dataset

In [4]:
import pandas as pd

# Ganti dengan Sheet ID dan Sheet Name jika perlu
sheet_id = "1GM9mlRGoUTNu0APh_J7v-tXZgUozNpInFOKiijA8B98"
sheet_name = "Sheet1"  # Ganti jika nama sheet kamu berbeda

# Buat URL dalam format export CSV
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

# Baca spreadsheet
df = pd.read_csv(url)

# Tambahkan kolom rating = 1 untuk semua reservasi
df["rating"] = 1

In [5]:
# 2. Encoding user & place
user_encoder = LabelEncoder()
place_encoder = LabelEncoder()
df['Nama'] = user_encoder.fit_transform(df['Nama'])
df['Tempat'] = place_encoder.fit_transform(df['Tempat'])

n_users = df['Nama'].nunique()
n_places = df['Tempat'].nunique()

# 3. Tambahkan rating implicit (1 berarti pernah reservasi)
df['rating'] = 1


In [6]:
# 4. Split data
X = df[['Nama', 'Tempat']]
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modelling

In [7]:
class MatrixFactorization(tf.keras.Model):
    def __init__(self, n_users, n_places, embedding_dim=32, **kwargs):
        super().__init__(**kwargs)
        self.n_users = n_users
        self.n_places = n_places
        self.embedding_dim = embedding_dim

        self.user_embedding = tf.keras.layers.Embedding(input_dim=n_users, output_dim=embedding_dim, name="user_embedding")
        self.place_embedding = tf.keras.layers.Embedding(input_dim=n_places, output_dim=embedding_dim, name="place_embedding")

    def call(self, inputs):
        user_vec = self.user_embedding(inputs[:, 0])
        place_vec = self.place_embedding(inputs[:, 1])
        dot_product = tf.reduce_sum(user_vec * place_vec, axis=1)
        return dot_product

    def get_config(self):
        return {
            "n_users": self.n_users,
            "n_places": self.n_places,
            "embedding_dim": self.embedding_dim
        }

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [12]:
# Buat model
# Ganti 'num_users' dan 'num_places' dengan 'n_users' dan 'n_places'
model = MatrixFactorization(n_users=df['Nama'].nunique(), n_places=df['Tempat'].nunique(), embedding_dim=32)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Training
# Ganti 'train_data' dan 'train_labels' dengan 'X_train' dan 'y_train'
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 1.0004 - mae: 1.0002 - val_loss: 0.9971 - val_mae: 0.9985
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9909 - mae: 0.9954 - val_loss: 0.9927 - val_mae: 0.9963
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9779 - mae: 0.9889 - val_loss: 0.9855 - val_mae: 0.9927
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9561 - mae: 0.9778 - val_loss: 0.9718 - val_mae: 0.9856
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9185 - mae: 0.9582 - val_loss: 0.9503 - val_mae: 0.9744
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8494 - mae: 0.9212 - val_loss: 0.9197 - val_mae: 0.9577
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.7616 - mae: 0.8718 - val_loss: 0.8744 - val_mae: 0.9317
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.6469 - mae: 0.8023 - val_loss: 0.8230 - val_mae: 0.8996
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5231 - mae:

In [13]:
# 7. Rekomendasi Top-3 Tempat per User
rekomendasi = []

for user_id_enc in range(n_users):
    # Tempat yang sudah dikunjungi user ini (encoded)
    visited = df[df['Nama'] == user_id_enc]['Tempat'].tolist()

    # Tempat yang belum dikunjungi
    candidates = [i for i in range(n_places) if i not in visited]

    # Buat pasangan input (user, place)
    input_pairs = np.array([[user_id_enc, pid] for pid in candidates], dtype=np.int32)

    # Prediksi skor untuk semua tempat kandidat
    scores = model.predict(input_pairs, verbose=0).reshape(-1)

    # Ambil 3 tempat teratas
    top_indices = scores.argsort()[-3:][::-1]
    top_place_encs = [candidates[i] for i in top_indices]
    top_places = place_encoder.inverse_transform(top_place_encs)

    rekomendasi.append({
        "user_id": user_encoder.inverse_transform([user_id_enc])[0],
        "rekomendasi": list(top_places)
    })

# Tambahan: Output global 3 tempat yang paling sering direkomendasikan
from collections import Counter

# Gabungkan semua rekomendasi jadi satu list
all_rekomendasi = sum([r["rekomendasi"] for r in rekomendasi], [])

# Hitung frekuensi dan ambil 3 teratas
top_places = Counter(all_rekomendasi).most_common(3)

# Tampilkan output akhir
print("\n3 Tempat yang Paling Sering Direkomendasikan untuk User:")
for i, (place, count) in enumerate(top_places, 1):
    print(f"{i}. {place} - {count} kali direkomendasikan")



3 Tempat yang Paling Sering Direkomendasikan untuk User:
1. Karaoke - 260 kali direkomendasikan
2. Aula - 257 kali direkomendasikan
3. Lahan Barat - 205 kali direkomendasikan


In [14]:
# Simpan model (.keras wajib agar load_model lancar)
model.save("model_rekomendasi.keras")